In [12]:
import os
import shutil
import random
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# =========================
# 1. DIVISÃO DOS DADOS
# =========================
original_data_dir = 'C:/Users/Magda/Classificador-CNN/kagglehub/datasets/ambarish/breakhis/versions/4/BreaKHis_v1/BreaKHis_v1/histology_slides/breast'
output_base = 'C:/Users/Magda/Classificador-CNN/kagglehub/datasets/ambarish/BreaKHis_split'


In [13]:

# =========================
# 2. PREPARAÇÃO DOS DADOS
# =========================
#img_width, img_height = 150, 150
#batch_size = 32

train_data_dir = os.path.join(output_base, 'train')
val_data_dir = os.path.join(output_base, 'val')
test_data_dir = os.path.join(output_base, 'test')

print(train_data_dir)
print(val_data_dir)
print(test_data_dir)

C:/Users/Magda/Classificador-CNN/kagglehub/datasets/ambarish/BreaKHis_split\train
C:/Users/Magda/Classificador-CNN/kagglehub/datasets/ambarish/BreaKHis_split\val
C:/Users/Magda/Classificador-CNN/kagglehub/datasets/ambarish/BreaKHis_split\test


In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Defina o tamanho das imagens
img_width, img_height = 244, 244  # Pode aumentar para 224x224 para mais detalhes

# Geradores de dados com Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
val_test_datagen = ImageDataGenerator(rescale=1. / 255)

# Ajuste os diretórios conforme seu projeto
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)



Found 5388 images belonging to 2 classes.
Found 1030 images belonging to 2 classes.
Found 1491 images belonging to 2 classes.


In [16]:
# Arquitetura do modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    GlobalAveragePooling2D(),
    Dropout(0.3),  # Dropout antes da densa
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),  # Regularização final
    Dense(train_generator.num_classes, activation='softmax')
])

# Compilação do modelo
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=1e-4),
    metrics=['accuracy']
)

checkpoint_path = 'C:/Users/Magda/Classificador-CNN/breast_cancer_best_model_untiled2.h5'

# 1. Carregar modelo treinado
model = load_model('C:/Users/Magda/Classificador-CNN/breast_cancer_best_model_untiled2.h5')

from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=True
)

# Callbacks para controle do treinamento
early_stop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-6)

# Treinamento do modelo
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=[model_checkpoint]
)


Epoch 1/100
169/169 [==============================] - 1084s 6s/step - loss: 0.1693 - accuracy: 0.9332 - val_loss: 0.2571 - val_accuracy: 0.8893
Epoch 2/100
169/169 [==============================] - 1105s 7s/step - loss: 0.1505 - accuracy: 0.9378 - val_loss: 0.4695 - val_accuracy: 0.8262
Epoch 3/100
169/169 [==============================] - 1319s 8s/step - loss: 0.1537 - accuracy: 0.9386 - val_loss: 0.1878 - val_accuracy: 0.9340
Epoch 4/100
169/169 [==============================] - 1250s 7s/step - loss: 0.1506 - accuracy: 0.9395 - val_loss: 0.5265 - val_accuracy: 0.8175
Epoch 5/100
169/169 [==============================] - 1630s 10s/step - loss: 0.1518 - accuracy: 0.9395 - val_loss: 0.1462 - val_accuracy: 0.9427
Epoch 6/100
169/169 [==============================] - 1607s 10s/step - loss: 0.1469 - accuracy: 0.9432 - val_loss: 0.1480 - val_accuracy: 0.9340
Epoch 7/100
169/169 [==============================] - 1605s 9s/step - loss: 0.1498 - accuracy: 0.9428 - val_loss: 0.1719 - val_

169/169 [==============================] - 1311s 8s/step - loss: 0.0877 - accuracy: 0.9699 - val_loss: 0.1526 - val_accuracy: 0.9427
Epoch 58/100
169/169 [==============================] - 1315s 8s/step - loss: 0.0789 - accuracy: 0.9692 - val_loss: 0.1423 - val_accuracy: 0.9456
Epoch 59/100
169/169 [==============================] - 1309s 8s/step - loss: 0.0917 - accuracy: 0.9653 - val_loss: 0.1605 - val_accuracy: 0.9515
Epoch 60/100
169/169 [==============================] - 1314s 8s/step - loss: 0.0888 - accuracy: 0.9664 - val_loss: 0.1299 - val_accuracy: 0.9583
Epoch 61/100
169/169 [==============================] - 1332s 8s/step - loss: 0.0810 - accuracy: 0.9696 - val_loss: 0.0636 - val_accuracy: 0.9748
Epoch 62/100
169/169 [==============================] - 1321s 8s/step - loss: 0.0700 - accuracy: 0.9735 - val_loss: 0.3585 - val_accuracy: 0.8816
Epoch 63/100
169/169 [==============================] - 1323s 8s/step - loss: 0.0875 - accuracy: 0.9681 - val_loss: 0.1240 - val_accuracy

In [9]:
model.save('C:/Users/Magda/Classificador-CNN/breast_cancer_best_model_untiled2.h5')  


In [10]:
# Avaliação no conjunto de teste
loss, accuracy = model.evaluate(test_generator)
print(f"Teste - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

47/47 [==============================] - 166s 4s/step - loss: 0.3625 - accuracy: 0.8390
Teste - Loss: 0.3625, Accuracy: 0.8390


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import os

# Caminho da imagem a ser testada
image_path = 'C:/Users/Magda/Classificador-CNN/kagglehub/datasets/ambarish/BreaKHis_split/test/benign/SOB/adenosis/SOB_B_A_14-22549AB/40X/SOB_B_A-14-22549AB-40-008.png'

# Parâmetros usados no treinamento
img_width, img_height = 128, 128  # ou o tamanho que você usou
class_indices = train_generator.class_indices  # Ou salve isso em um dicionário separado
class_labels = dict((v, k) for k, v in class_indices.items())

# 1. Carregar modelo treinado
model = load_model('C:/Users/Magda/Classificador-CNN/breast_cancer_best_model_untiled2.h5')

# 2. Carregar e processar imagem
img = image.load_img(image_path, target_size=(img_width, img_height))
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Normalização igual à do treinamento
img_array = np.expand_dims(img_array, axis=0)  # Adiciona dimensão do batch

# 3. Fazer predição
prediction = model.predict(img_array)
predicted_class_index = np.argmax(prediction[0])
predicted_class_label = class_labels[predicted_class_index]
confidence = prediction[0][predicted_class_index]

# 4. Exibir resultado
plt.imshow(img)
plt.axis('off')
plt.title(f'Classe: {predicted_class_label} ({confidence * 100:.2f}%)')
plt.show()


NameError: name 'train_generator' is not defined

In [ ]:

# =========================
# 6. AVALIAÇÃO E MÉTRICAS
# =========================
predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Matriz de confusão
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
plt.title("Matriz de Confusão - Conjunto de Teste")
plt.tight_layout()

conf_matrix_path = 'C:/Users/Magda/Classificador-CNN/matriz_confusao.png'
plt.savefig(conf_matrix_path)
print(f"✅ Matriz de confusão salva em: {conf_matrix_path}")

# Relatório de classificação
report = classification_report(y_true, y_pred, target_names=class_labels)
report_path = 'C:/Users/Magda/Classificador-CNN/relatorio_classificacao.txt'
with open(report_path, 'w') as f:
    f.write("Relatório de Classificação - Conjunto de Teste\n\n")
    f.write(report)
print(f"✅ Relatório salvo em: {report_path}")

# =========================
# 7. CURVAS DE TREINAMENTO
# =========================
plt.figure(figsize=(12, 5))

# Acurácia
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treinamento')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia por Época')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

# Perda
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treinamento')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda por Época')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()

# Salvar gráfico
plot_path = 'C:/Users/Magda/Classificador-CNN/curvas_treinamento.png'
plt.tight_layout()
plt.savefig(plot_path)
print(f"✅ Curvas de treinamento salvas em: {plot_path}")